# Day 44: Anomaly Detection Monitor

In this lab, we implement a **Real-time Anomaly Detection Monitor**.
We use statistical baselining (Z-score) to detect when a metric (like API latency) deviates significantly from the recent norm.

This helps catch issues like:
- **DDoS Attacks** (Latency spikes)
- **Configuration Errors** (Suddenly constant values)
- **Infinite Loops** (High resource usage)

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

# Add root directory to sys.path
sys.path.append(os.path.abspath('../../'))

from src.observability.anomaly_monitor import RuntimeAnomalyMonitor

## 1. Simulate Latency Stream

We generate normal traffic (Gaussian) and inject a spike.

In [ ]:
np.random.seed(42)

# 100 normal requests: Mean=50ms, Std=5ms
normal_latency = np.random.normal(50, 5, 100)

# 10 anomaly requests: Mean=200ms (Spike)
spike_latency = np.random.normal(200, 10, 10)

# Combine
stream = np.concatenate([normal_latency, spike_latency, normal_latency])

## 2. Run Monitor

We feed the stream into the monitor one by one.

In [ ]:
monitor = RuntimeAnomalyMonitor(window_size=30, z_threshold=3.0)

anomalies = []
values = []

for i, val in enumerate(stream):
    # Check first
    result = monitor.check_anomaly(val)
    
    if result['is_anomaly']:
        print(f"[ALERT] Anomaly at index {i}: Value={val:.2f}, Z={result['z_score']:.2f}")
        anomalies.append(i)
        
    # Update history
    monitor.update(val)
    values.append(val)

## 3. Visualize

Plot the latency stream and mark anomalies.

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(values, label='Latency (ms)')
plt.scatter(anomalies, [values[i] for i in anomalies], color='red', label='Detected Anomaly')
plt.axhline(y=50, color='g', linestyle='--', alpha=0.5, label='Normal Mean')
plt.title("Latency Stream with Anomaly Detection")
plt.xlabel("Request ID")
plt.ylabel("Latency (ms)")
plt.legend()
plt.show()